#### Import packages

In [1]:
import keras
import pandas as pd
import numpy as np
from keras.datasets import mnist


from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from keras.models import Sequential
from keras.utils import np_utils
from keras.layers import Dense, Dropout, GaussianNoise, Conv1D
from keras.preprocessing.image import ImageDataGenerator

import matplotlib.pyplot as plt

Using TensorFlow backend.


#### Data import and clean

In [2]:
# read in mnist dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# normalize data
x_train = x_train/255
x_test = x_test/255

# get flat datasets for non-conv tasks
x_train_flat = x_train.reshape(60000, 28*28)
x_test_flat = x_test.reshape(10000, 28*28)

# one hot encode targets
y_train = keras.utils.to_categorical(y_train, num_classes=None, dtype='float32')
y_test = keras.utils.to_categorical(y_test, num_classes=None, dtype='float32')

#### Model stuff

In [3]:
lot_size = 600

In [4]:
## PCA LAYER
n_components = 60

pca = PCA(n_components=n_components)
x_pca_train = pca.fit_transform(x_train_flat)
x_pca_test = pca.transform(x_test_flat)
pca_std = np.std(x_pca_train)

In [5]:
# Neural Net building on PCA
model = Sequential()
layers = 1
units = 1000

model.add(Dense(units, input_dim=n_components, activation='relu'))
model.add(Dense(10, activation = 'softmax'))

model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['categorical_accuracy'])

In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1000)              61000     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                10010     
Total params: 71,010
Trainable params: 71,010
Non-trainable params: 0
_________________________________________________________________


In [7]:
model.fit(x_pca_train, y_train, epochs=100, batch_size=lot_size, validation_split=0.2, verbose=2)

Train on 48000 samples, validate on 12000 samples
Epoch 1/100
 - 1s - loss: 0.4942 - categorical_accuracy: 0.8916 - val_loss: 0.2087 - val_categorical_accuracy: 0.9429
Epoch 2/100
 - 0s - loss: 0.1744 - categorical_accuracy: 0.9509 - val_loss: 0.1382 - val_categorical_accuracy: 0.9621
Epoch 3/100
 - 0s - loss: 0.1180 - categorical_accuracy: 0.9680 - val_loss: 0.1082 - val_categorical_accuracy: 0.9700
Epoch 4/100
 - 0s - loss: 0.0882 - categorical_accuracy: 0.9757 - val_loss: 0.0926 - val_categorical_accuracy: 0.9749
Epoch 5/100
 - 0s - loss: 0.0689 - categorical_accuracy: 0.9814 - val_loss: 0.0816 - val_categorical_accuracy: 0.9780
Epoch 6/100
 - 0s - loss: 0.0558 - categorical_accuracy: 0.9855 - val_loss: 0.0761 - val_categorical_accuracy: 0.9792
Epoch 7/100
 - 1s - loss: 0.0459 - categorical_accuracy: 0.9886 - val_loss: 0.0709 - val_categorical_accuracy: 0.9798
Epoch 8/100
 - 0s - loss: 0.0384 - categorical_accuracy: 0.9905 - val_loss: 0.0681 - val_categorical_accuracy: 0.9803
Epoch 

Epoch 69/100
 - 1s - loss: 4.5446e-07 - categorical_accuracy: 1.0000 - val_loss: 0.1136 - val_categorical_accuracy: 0.9841
Epoch 70/100
 - 0s - loss: 4.2400e-07 - categorical_accuracy: 1.0000 - val_loss: 0.1142 - val_categorical_accuracy: 0.9843
Epoch 71/100
 - 0s - loss: 4.0047e-07 - categorical_accuracy: 1.0000 - val_loss: 0.1138 - val_categorical_accuracy: 0.9839
Epoch 72/100
 - 0s - loss: 3.7841e-07 - categorical_accuracy: 1.0000 - val_loss: 0.1139 - val_categorical_accuracy: 0.9842
Epoch 73/100
 - 0s - loss: 3.6064e-07 - categorical_accuracy: 1.0000 - val_loss: 0.1146 - val_categorical_accuracy: 0.9842
Epoch 74/100
 - 0s - loss: 3.4471e-07 - categorical_accuracy: 1.0000 - val_loss: 0.1148 - val_categorical_accuracy: 0.9841
Epoch 75/100
 - 0s - loss: 3.3260e-07 - categorical_accuracy: 1.0000 - val_loss: 0.1151 - val_categorical_accuracy: 0.9841
Epoch 76/100
 - 0s - loss: 3.1771e-07 - categorical_accuracy: 1.0000 - val_loss: 0.1157 - val_categorical_accuracy: 0.9841
Epoch 77/100
 - 

In [8]:
model.evaluate(x_pca_test, y_test)

10000/10000 [==============================] - 0s 15us/step


[0.11146881464393714, 0.9843]

#### Import DP optimizers - straight from tutorial

In [2]:
# import os
# os.chdir('../../../privacy')
# os.getcwd()
from privacy import analysis
import sys

In [3]:
import tensorflow as tf
from privacy.analysis.rdp_accountant import compute_rdp
from privacy.analysis.rdp_accountant import get_privacy_spent
from privacy.dp_query.gaussian_query import GaussianAverageQuery
from privacy.optimizers.dp_optimizer import DPGradientDescentOptimizer

In [4]:
tf.VERSION

'1.12.0'

In [5]:
# Compatibility with tf 1 and 2 APIs
try:
    GradientDescentOptimizer = tf.train.GradientDescentOptimizer
except:  # pylint: disable=bare-except
    GradientDescentOptimizer = tf.optimizers.SGD  # pylint: disable=invalid-name

In [6]:
# Fix for FLAGS returning UnrecognizedFlagError - 
# https://stackoverflow.com/questions/48198770/tensorflow-1-5-0-rc0-error-using-tf-app-flags

FLAGS = tf.flags.FLAGS
remaining_args = FLAGS([sys.argv[0]] + [flag for flag in sys.argv if flag.startswith("--")])
assert(remaining_args == [sys.argv[0]]) 

tf.flags.DEFINE_boolean('dpsgd', True, 'If True, train with DP-SGD. If False, '
                        'train with vanilla SGD.')
tf.flags.DEFINE_float('learning_rate', 0.15, 'Learning rate for training')
tf.flags.DEFINE_float('noise_multiplier', 1.1,
                      'Ratio of the standard deviation to the clipping norm')
tf.flags.DEFINE_float('l2_norm_clip', 1.0, 'Clipping norm')
tf.flags.DEFINE_integer('batch_size', 250, 'Batch size')
tf.flags.DEFINE_integer('epochs', 60, 'Number of epochs')
tf.flags.DEFINE_integer('microbatches', 250, 'Number of microbatches '
                        '(must evenly divide batch_size)')
tf.flags.DEFINE_string('model_dir', None, 'Model directory')

In [7]:
FLAGS.batch_size

250

In [8]:
def load_mnist():
    """Loads MNIST and preprocesses to combine training and validation data."""
    train, test = tf.keras.datasets.mnist.load_data()
    train_data, train_labels = train
    test_data, test_labels = test

    train_data = np.array(train_data, dtype=np.float32) / 255
    test_data = np.array(test_data, dtype=np.float32) / 255

    train_data = train_data.reshape(train_data.shape[0], 28, 28, 1)
    test_data = test_data.reshape(test_data.shape[0], 28, 28, 1)

    train_labels = np.array(train_labels, dtype=np.int32)
    test_labels = np.array(test_labels, dtype=np.int32)

    train_labels = tf.keras.utils.to_categorical(train_labels, num_classes=10)
    test_labels = tf.keras.utils.to_categorical(test_labels, num_classes=10)

    assert train_data.min() == 0.
    assert train_data.max() == 1.
    assert test_data.min() == 0.
    assert test_data.max() == 1.

    return train_data, train_labels, test_data, test_labels

In [9]:
# def main(unused_argv):
tf.logging.set_verbosity(tf.logging.INFO)
if FLAGS.batch_size % FLAGS.microbatches != 0:
    raise ValueError('Number of microbatches should divide evenly batch_size')

In [10]:
# Load training and test data.
train_data, train_labels, test_data, test_labels = load_mnist()

In [12]:
# Define a sequential Keras model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, 8,
                            strides=2,
                            padding='same',
                            activation='relu',
                            input_shape=(28, 28, 1)),
    
    tf.keras.layers.MaxPool2D(2, 1),
    tf.keras.layers.Conv2D(32, 4,
                             strides=2,
                             padding='valid',
                             activation='relu'),
    
    tf.keras.layers.MaxPool2D(2, 1),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(10)
  ])

In [19]:
if FLAGS.dpsgd:
    dp_average_query = GaussianAverageQuery(
                                            FLAGS.l2_norm_clip,
                                            FLAGS.l2_norm_clip * FLAGS.noise_multiplier,
                                            FLAGS.microbatches)
    optimizer = DPGradientDescentOptimizer(
                                            dp_average_query,
                                            FLAGS.microbatches,
                                            learning_rate=FLAGS.learning_rate,
                                            unroll_microbatches=True)
    
    # Compute vector of per-example loss rather than its mean over a minibatch.
    loss = tf.keras.backend.categorical_crossentropy(
                                            from_logits=True, reduction=tf.losses.Reduction.NONE)

else:
    

#     tensorflow/tensorflow/contrib/keras/python/keras/backend.py
    
    optimizer = GradientDescentOptimizer(learning_rate=FLAGS.learning_rate)
    loss = tf.keras.backend.categorical_crossentropy(from_logits=True)

    # Compile model with Keras
    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

    # Train model with Keras
    model.fit(train_data, train_labels,
                        epochs=FLAGS.epochs,
                        validation_data=(test_data, test_labels),
                        batch_size=FLAGS.batch_size)

TypeError: categorical_crossentropy() got an unexpected keyword argument 'reduction'

In [16]:
tf.keras.losses.categorical_crossentropy()

<function tensorflow.python.keras.losses.categorical_crossentropy(y_true, y_pred)>

In [ ]:
  # Compute the privacy budget expended.
  if FLAGS.noise_multiplier == 0.0:
    print('Trained with vanilla non-private SGD optimizer')
  orders = [1 + x / 10. for x in range(1, 100)] + list(range(12, 64))
  sampling_probability = FLAGS.batch_size / 60000
  rdp = compute_rdp(q=sampling_probability,
                    noise_multiplier=FLAGS.noise_multiplier,
                    steps=(FLAGS.epochs * 60000 // FLAGS.batch_size),
                    orders=orders)
  # Delta is set to 1e-5 because MNIST has 60000 training points.
  eps = get_privacy_spent(orders, rdp, target_delta=1e-5)[0]
  print('For delta=1e-5, the current epsilon is: %.2f' % eps)